In [1]:
# from IPython.display import YouTubeVideo
# YouTubeVideo('To be Announced')

# 如何使用和开发微信聊天机器人的系列教程
# A workshop to develop & use an intelligent and interactive chat-bot in WeChat

### WeChat is a popular social media app, which has more than 800 million monthly active users.

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: right;">

### http://www.KudosData.com

by: Sam.Gu@KudosData.com


May 2017 ========== Scan the QR code to become trainer's friend in WeChat ========>>

### 第五课：视频识别和处理
### Lesson 5: Video Recognition & Processing
* 识别视频的场景片段 (Detect shots change in video)
* 识别视频消息中的物体名字 (Recognize objects in video/shots)
* 直接搜索视频内容 (Search content in video)

### Using Google Cloud Platform's Machine Learning APIs

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Cloud Vision API </li>
<li> Google Cloud Speech API </li>
<li> Google Cloud Translate API </li>
<li> Google Cloud Natural Language API </li>
<li> Google Cloud Video API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [2]:
# Copyright 2016 Google Inc.
# Licensed under the Apache License, Version 2.0 (the "License"); 
# !pip install --upgrade google-api-python-client

### 导入需要用到的一些功能程序库：

In [3]:
import io, os, subprocess, sys, time, datetime, requests, itchat, json, numpy as np
from itchat.content import *
from googleapiclient.discovery import build
from IPython.display import HTML
# Python 2
if sys.version_info[0] < 3:
    import urllib2
# Python 3
else:
    import urllib.request

### Using Google Cloud Platform's Machine Learning APIs

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [4]:
# Here I read in my own API_KEY from a file, which is not shared in Github repository:
with io.open('../../../API_KEY.txt') as fp: 
    for line in fp: APIKEY = line

# You need to un-comment below line and replace 'APIKEY' variable with your own GCP API key:
# APIKEY='AIzaSyCvxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [5]:
# Below is for GCP Video API
# video_service = build('videointelligence', 'v1', developerKey=APIKEY)
video_service = build('videointelligence', 'v1beta1', developerKey=APIKEY)
# check video processing progress
video_operation_service = build('videointelligence', 'v1', developerKey=APIKEY)

### 多媒体文件的二进制base64码转换 (Define media pre-processing functions)

In [6]:
# Import the base64 encoding library.
import base64
# Pass the media data to an encoding function.
def encode_media(media_file):
    with io.open(media_file, "rb") as media_file:
        media_content = media_file.read()
# Python 2
    if sys.version_info[0] < 3:
        return base64.b64encode(media_content).decode('ascii')
# Python 3
    else:
        return base64.b64encode(media_content).decode('utf-8')

### 机器智能API接口控制参数 (Define control parameters for API)

In [7]:
# API control parameter for 视频识别和处理 Video Recognition & Processing
parm_video_api_features = ['LABEL_DETECTION', 'SHOT_CHANGE_DETECTION']
# parm_video_api_features = ['LABEL_DETECTION']
# parm_video_api_features = ['SHOT_CHANGE_DETECTION']
parm_video_api_display_seconds = True # Ture: use seconds for display; False: use microseconds to display video shot duration
parm_video_response = {} # global variable

### 定义一个发起视频处理请求的小功能
Define utility function to call Video API:

In [8]:
# Invoke Video API

def KudosData_VIDEO_DETECTION(video_base64):
    print ('[INFOR] Start Video Analysis...')
##########################################################################################
# 1. Send request for video processing
##########################################################################################
    request1 = video_service.videos().annotate(body={
#               "inputUri": string,
              "inputContent": video_base64,
              "features": parm_video_api_features, # 'LABEL_DETECTION' & 'SHOT_CHANGE_DETECTION'
        })
    responses1 = request1.execute(num_retries=3)
    
    if 'name' in responses1: 
        print ('Internal video/operation name assinged : %s' % responses1['name'])
##########################################################################################
# 2. Check progress till completion (Video requires asynchronous long processing...)
##########################################################################################
        responses2 = KudosData_VIDEO_DETECTION_CHECK_PROCRESS(responses1['name'], APIKEY)
    else:
        print ('[ERROR] Calling Video API request failed. Please re-try.')
    
    return responses2

### 定义一个查询视频处理进展的小功能 Define utility function to check progress of Video API:

因为视频文件通常较大，处理费时。谷歌API接口采用了异步处理方式，所以会有此步骤。

In [9]:
# Keep checking progress of Video API processing, till full completion.

def KudosData_VIDEO_DETECTION_CHECK_PROCRESS(name, apikey):
    flag_completion = False
    operation_url = 'https://videointelligence.googleapis.com/v1/operations/' + name + '?key=' + apikey
#     print('operation_url : ', operation_url)
    while not flag_completion:
        # Python 2
        if sys.version_info[0] < 3:
            response_operation = json.loads(urllib2.urlopen(operation_url).read())
        # Python 3
        else:
#             response_operation = json.loads(urllib.request.urlopen(operation_url).read())
            response_operation = json.loads(urllib.request.urlopen(operation_url).read().decode('utf-8'))
        
        # define an array, containing 'progress %' of each Video API
        video_api_progress_pct = np.zeros(shape=(1, len(response_operation['metadata']['annotationProgress'])))
#         print('... length : %d' % len(response_operation['metadata']['annotationProgress']))
        for i in range(len(response_operation['metadata']['annotationProgress'])):
            if 'progressPercent' in response_operation['metadata']['annotationProgress'][i]:
                video_api_progress_pct[0][i] = response_operation['metadata']['annotationProgress'][i]['progressPercent']
#                 print ('>>> Video API {} Progress : {} %'.format(i+1, video_api_progress_pct[0][i]))
        print ('  overall progress : {} %'.format(np.mean(video_api_progress_pct[0])))
        if min(video_api_progress_pct[0]) == 100: 
            flag_completion = True
            print ('  Video API Completed !\n')

#         time.sleep(5)
        time.sleep(2)
#         print('... time.sleep')

    return response_operation

<font color='blue'>< Start of interactive demo ></font>

### 导入测试短片 Import video:

In [10]:
video_file = 'reference/video.mp4'
# video_file = 'reference/SampleVideo_360x240_2mb.mp4'

In [11]:
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
         .format(encode_media(video_file)))

### 正式开始视频处理 Here we call Video API through our defined function:

In [13]:
# Send API request & Obtain results:
parm_video_response = KudosData_VIDEO_DETECTION(encode_media(video_file))

# Obtain results, without resending API request:
# parm_video_response = KudosData_VIDEO_DETECTION_CHECK_PROCRESS('asia-east1.11590064400054192243', APIKEY)

[INFOR] Start Video Analysis...
Internal video/operation name assinged : asia-east1.10958220561835718202
  overall progress : 100.0 %
  Video API Completed !



### 查看原始格式的结果 View raw results:

In [14]:
parm_video_response

# metadata in response: Version of Video API used; Differenc Video APIs called, and start/end processing time.
# parm_video_response['metadata']

# actual video API detecion results in response:
# parm_video_response['response']['annotationResults'][0]

{'done': True,
 'metadata': {'@type': 'type.googleapis.com/google.cloud.videointelligence.v1beta1.AnnotateVideoProgress',
  'annotationProgress': [{'progressPercent': 100,
    'startTime': '2017-06-07T08:09:01.043907Z',
    'updateTime': '2017-06-07T08:09:03.390992Z'},
   {'progressPercent': 100,
    'startTime': '2017-06-07T08:09:01.043907Z',
    'updateTime': '2017-06-07T08:09:04.187755Z'}]},
 'name': 'asia-east1.10958220561835718202',
 'response': {'@type': 'type.googleapis.com/google.cloud.videointelligence.v1beta1.AnnotateVideoResponse',
  'annotationResults': [{'labelAnnotations': [{'description': 'LEGO',
      'languageCode': 'en-us',
      'locations': [{'confidence': 0.9779057,
        'level': 'VIDEO_LEVEL',
        'segment': {'endTimeOffset': '-1', 'startTimeOffset': '-1'}},
       {'confidence': 0.9779057,
        'level': 'SHOT_LEVEL',
        'segment': {'endTimeOffset': '5499941'}}]},
     {'description': 'Machine',
      'languageCode': 'en-us',
      'locations': [{'c

### * 识别视频的场景片段 (Detect shots change in video)

In [15]:
# parm_video_response['response']['annotationResults'][0]['shotAnnotations']

In [16]:
if parm_video_api_display_seconds: # our pre-defined control parm
    parm_denominator = 1000000 # dispay results in seconds
else:
    parm_denominator = 1  # dispay results in microseconds

print(u'[ 视频场景片段 ]')
print('Number of scenery/shots detected : {}'.format(len(parm_video_response['response']['annotationResults'][0]\
                                                         ['shotAnnotations'])))
for i in range(len(parm_video_response['response']['annotationResults'][0]['shotAnnotations'])):
#     print(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]) 
    if i == 0:
        print('Shot 1 : 0 ~ {} {}'
            .format(
                int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]\
                                        ['endTimeOffset'])/parm_denominator
              , 'seconds' if parm_video_api_display_seconds else 'microseconds'
            ))
    else:
        print('Shot {} : {} ~ {} {}'
            .format(i+1
              , int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]\
                                        ['startTimeOffset'])/parm_denominator
              , int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]\
                                        ['endTimeOffset'])/parm_denominator
              , 'seconds' if parm_video_api_display_seconds else 'microseconds'
            ))

[ 视频场景片段 ]
Number of scenery/shots detected : 1
Shot 1 : 0 ~ 5.499941 seconds


### * 识别视频消息中的物体名字 (Recognize objects in video/shots)

In [17]:
# parm_video_response['response']['annotationResults'][0]['labelAnnotations']

In [18]:
print(u'[ 视频物体识别 ]')
print('Number of objects detected : {}'.format(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])))
print('')
for i in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])):
    # print each object's name:
    print('Object {} : {}'.format(
              i+1
            , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description']
        ))

[ 视频物体识别 ]
Number of objects detected : 7

Object 1 : LEGO
Object 2 : Machine
Object 3 : Robot
Object 4 : Stop motion
Object 5 : The Lego Group
Object 6 : Toy
Object 7 : Toy block


In [19]:
print(u'[ 视频物体识别 ]')
print('Number of objects detected : {}'.format(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])))

for i in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])):
    # print each object's name:
    print('')
    print('Object {} : {}\n    @  {} Locations: '.format(
              i+1
            , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description']
            , len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
        ))
    
        # print each object's related shot/location in video:
    for j in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])):
#         print(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j])
        print('   [{}] Confidence : {} | {} | {} '.format(
              j+1
            , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['confidence']
            , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['segment']
            , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['level']
              ))
  

[ 视频物体识别 ]
Number of objects detected : 7

Object 1 : LEGO
    @  2 Locations: 
   [1] Confidence : 0.9779057 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
   [2] Confidence : 0.9779057 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 2 : Machine
    @  2 Locations: 
   [1] Confidence : 0.4638117 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
   [2] Confidence : 0.4638117 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 3 : Robot
    @  2 Locations: 
   [1] Confidence : 0.71584797 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
   [2] Confidence : 0.71584797 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 4 : Stop motion
    @  2 Locations: 
   [1] Confidence : 0.40495867 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
   [2] Confidence : 0.40495867 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 5 : The Lego Group
    @  2 Locations: 
   [1] Confidence : 0.654321 | {'startTimeOffset': '-1', 'e

### * 直接搜索视频内容 (Search content in video)
使用<font color='blue'>**文字**</font>关键词对视频<font color='blue'>**内容**</font>进行搜索

In [20]:
# Define utility function for video search

# return a list of shot locations, based on list of object name
def KudosData_search_location(list_object_name):   
#     print('Look for object : %s' % object_name)

    for i in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])):
        if parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description'] == list_object_name[0]: 
#             print('Found  {} Locations   :'.format(
#                     len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
#                 ))
            return parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations']

    return [] # If not found, return empty list []

### simple score integration: Certainty Factor
### CF = CF1 + CF2 * ( 1 - CF1 )
def KudosData_CF(cf1, cf2): return cf1 + cf2 * ( 1 - cf1)

# return a list of shot-locations, based on matched shot-loactions of two objects
def KudosData_match_location(list_location_1, list_location_2):   
    list_location_matched = []
    for i in range(len(list_location_1)):
        for j in range(len(list_location_2)):
            if list_location_1[i]['level'] == list_location_2[j]['level'] and \
               list_location_1[i]['segment'] == list_location_2[j]['segment']: 
#                 print('Shots matched: [i={}] [j={}]'.format(i, j))
                dict_location_matched = list_location_2[j].copy()
                dict_location_matched['confidence'] = KudosData_CF(list_location_1[i]['confidence'], 
                                                                   list_location_2[j]['confidence'])
                list_location_matched.append(dict_location_matched)
#     print(list_location_matched)
    return list_location_matched

def KudosData_search(list_object_name):
    if len(list_object_name) == 1:
        return KudosData_search_location([list_object_name[0]])    
    else:
        return KudosData_match_location(KudosData_search_location([list_object_name[0]]), 
                                        KudosData_search_location([list_object_name[1]]))
    

### 搜索示例 Demo:
单关键词

In [21]:
KudosData_search(['Toy'])

[{'confidence': 0.9878092,
  'level': 'VIDEO_LEVEL',
  'segment': {'endTimeOffset': '-1', 'startTimeOffset': '-1'}},
 {'confidence': 0.9878092,
  'level': 'SHOT_LEVEL',
  'segment': {'endTimeOffset': '5499941'}}]

In [23]:
KudosData_search(['Robot'])

[{'confidence': 0.71584797,
  'level': 'VIDEO_LEVEL',
  'segment': {'endTimeOffset': '-1', 'startTimeOffset': '-1'}},
 {'confidence': 0.71584797,
  'level': 'SHOT_LEVEL',
  'segment': {'endTimeOffset': '5499941'}}]

In [ ]:
KudosData_search(['Animal'])

In [ ]:
KudosData_search(['Sky'])

### 多关键词查找 Joint search by keywords: **'Animal' & 'Sky'**

[ 搜索视频内容 ] **'Animal'**
*   [2] Confidence : 0.89243287 | SHOT_LEVEL | {u'endTimeOffset': u'8333309'} 
*   [5] Confidence : **0.8764819** | SHOT_LEVEL | <font color='blue'>{u'endTimeOffset': u'23533281', u'startTimeOffset': u'21733318'}</font>
*   [1] Confidence : 0.8764819 | VIDEO_LEVEL | {u'endTimeOffset': u'-1', u'startTimeOffset': u'-1'}
*   [6] Confidence : 0.82209057 | SHOT_LEVEL | {u'endTimeOffset': u'25666689', u'startTimeOffset': u'23599970'} 
*   [4] Confidence : 0.7916033 | SHOT_LEVEL | {u'endTimeOffset': u'15999984', u'startTimeOffset': u'13000003'} 
*   [3] Confidence : **0.4363379** | SHOT_LEVEL | <font color='blue'>{u'endTimeOffset': u'12933314', u'startTimeOffset': u'8399998'}</font>

[ 搜索视频内容 ] **'Sky'**
*   [1] Confidence : **0.81967264** | SHOT_LEVEL | <font color='blue'>{u'endTimeOffset': u'12933314', u'startTimeOffset': u'8399998'}</font>
*   [2] Confidence : **0.496** | SHOT_LEVEL | <font color='blue'>{u'endTimeOffset': u'23533281', u'startTimeOffset': u'21733318'}</font>


### 对于多关键词，怎样找出最相关的视频片段？ How to rank the video shot relevance for a combination of keywords?
A simpel method is to combine confidence pair by 'Certainty Factor': CF = Confidence1 + Confidence2 * ( 1 - Confidence1 )


In [ ]:
# CF = 0.81967264 + 0.4363379 * ( 1 - 0.81967264 ) = ?
print('CF (combined confidence score) = {}'.format(0.81967264 + 0.4363379 * ( 1 - 0.81967264 )))

In [ ]:
# * CF = 0.496 + 0.8764819 * ( 1 - 0.496 ) = ?
print('CF (combined confidence score) = {}'.format(0.496 + 0.8764819 * ( 1 - 0.496 )))

### 搜索示例 Demo:
多关键词

In [22]:
KudosData_search(['Toy', 'Robot'])

[{'confidence': 0.996535959432676,
  'level': 'VIDEO_LEVEL',
  'segment': {'endTimeOffset': '-1', 'startTimeOffset': '-1'}},
 {'confidence': 0.996535959432676,
  'level': 'SHOT_LEVEL',
  'segment': {'endTimeOffset': '5499941'}}]

In [ ]:
KudosData_search(['Animal', 'Sky'])

In [ ]:
KudosData_search(['Animal', 'Grassland'])

<font color='blue'>
[提问] 如果是三个或更多关键词呢，怎么处理？ 
[Question] Which video shot is most relvant for: Animal, Sky, and Grassland? And why?
</font>

In [ ]:
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
     .format(encode_media(video_file)))

In [ ]:
KudosData_search(['Animal', 'SamGu'])

<font color='blue'>< End of interactive demo ></font>

### 定义一个输出为视频分析结果的文本消息的小功能，用于微信回复：

In [24]:
def KudosData_video_generate_reply(parm_video_response):
    video_reply = u'[ Video Analysis 视频分析结果 ]'
    
    # 1. 识别视频的场景片段 (Detect shots change in video)
    if parm_video_api_display_seconds: # our pre-defined control parm
        parm_denominator = 1000000 # dispay results in seconds
    else:
        parm_denominator = 1  # dispay results in microseconds

    video_reply += '\n' + (u'\n[ 视频场景片段 ]')
    video_reply += '\n' + ('Number of scenery/shot detected : {}'
                           .format(len(parm_video_response['response']['annotationResults'][0]['shotAnnotations'])))
    for i in range(len(parm_video_response['response']['annotationResults'][0]['shotAnnotations'])):
    #     video_reply += (parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]) 
        if i == 0:
            video_reply += '\n' + ('Shot 1 : 0 ~ {} {}'
                .format(
                    int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]['endTimeOffset'])
                    /parm_denominator
                  , 'seconds' if parm_video_api_display_seconds else 'microseconds'
                ))
        else:
            video_reply += '\n' + ('Shot {} : {} ~ {} {}'
                .format(i+1
                  , int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]['startTimeOffset'])
                        /parm_denominator
                  , int(parm_video_response['response']['annotationResults'][0]['shotAnnotations'][i]['endTimeOffset'])
                        /parm_denominator
                  , 'seconds' if parm_video_api_display_seconds else 'microseconds'
                ))

    
    # 2. 识别视频消息中的物体名字 (Recognize objects in video/shots)
    
    video_reply += '\n' + (u'\n[ 视频物体识别 ]')
    video_reply += '\n' + ('Number of objects detected : {}'
                    .format(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])))

    for i in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'])):
        # video_reply +=  each object's name:
        video_reply += '\n'
        video_reply += '\n' + ('Object {} : {}\n    @  {} Locations: '.format(
                  i+1
                , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['description']
                , len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])
            ))

            # video_reply +=  each object's location in video:
        for j in range(len(parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'])):
    #         video_reply += (parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j])
            video_reply += '\n' + ('      [{}] Confidence : {} | {} | {} '.format(
                  j+1
                , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['confidence']
                , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['segment']
                , parm_video_response['response']['annotationResults'][0]['labelAnnotations'][i]['locations'][j]['level']
                  ))

    return video_reply

In [25]:
print(KudosData_video_generate_reply(parm_video_response))

[ Video Analysis 视频分析结果 ]

[ 视频场景片段 ]
Number of scenery/shot detected : 1
Shot 1 : 0 ~ 5.499941 seconds

[ 视频物体识别 ]
Number of objects detected : 7

Object 1 : LEGO
    @  2 Locations: 
      [1] Confidence : 0.9779057 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
      [2] Confidence : 0.9779057 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 2 : Machine
    @  2 Locations: 
      [1] Confidence : 0.4638117 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
      [2] Confidence : 0.4638117 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 3 : Robot
    @  2 Locations: 
      [1] Confidence : 0.71584797 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
      [2] Confidence : 0.71584797 | {'endTimeOffset': '5499941'} | SHOT_LEVEL 

Object 4 : Stop motion
    @  2 Locations: 
      [1] Confidence : 0.40495867 | {'startTimeOffset': '-1', 'endTimeOffset': '-1'} | VIDEO_LEVEL 
      [2] Confidence : 0.40495867 | {'endTimeOffset': '54

### 用微信App扫QR码图片来自动登录

In [26]:
itchat.auto_login(hotReload=True) # hotReload=True: 退出程序后暂存登陆状态。即使程序关闭，一定时间内重新开启也可以不用重新扫码。
# itchat.auto_login(enableCmdQR=-2) # enableCmdQR=-2: 命令行显示QR图片

Getting uuid of QR code.
Please scan the QR code to log in.


www.KudosData.com : QR.png


Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 酷豆陪聊妹


In [27]:
# @itchat.msg_register([VIDEO], isGroupChat=True)
@itchat.msg_register([VIDEO])
def download_files(msg):
    msg.download(msg.fileName)
    print('\nDownloaded video file name is: %s' % msg['FileName'])
    ##############################################################################################################
    #                                          call video analysis APIs                                          #
    ##############################################################################################################
    global parm_video_response # save into global variable, which can be accessed by next WeChat keyword search
    parm_video_response = KudosData_VIDEO_DETECTION(encode_media(msg['FileName']))
    ##############################################################################################################
    #                                          format video API results                                          #
    ##############################################################################################################
    video_analysis_reply = KudosData_video_generate_reply(parm_video_response)
    print ('')
    print(video_analysis_reply)
    return video_analysis_reply

In [28]:
# 单聊模式，基于关键词进行视频搜索：
@itchat.msg_register([TEXT])
def text_reply(msg):
#     if msg['isAt']:
        list_keywords = [x.strip() for x in msg['Text'].split(',')]
        # call video search function:
        search_responses = KudosData_search(list_keywords) # return is a list
        # Format search results:
        search_reply = '[ Video Search 视频搜索结果 ]' + '\n'
        if len(search_responses) == 0:
            search_reply += '[ Nill 无结果 ]'
        else:
            for i in range(len(search_responses)): search_reply += '\n' + str(search_responses[i])
        print ('')
        print (search_reply)
        return search_reply

In [ ]:
# 群聊模式，如果收到 @ 自己的文字信息，基于关键词进行视频搜索：
@itchat.msg_register([TEXT])
def text_reply_group(msg):
    if msg['isAt']:
        list_keywords = [x.strip() for x in msg['Text'].split(',')]
        # call video search function:
        search_responses = KudosData_search(list_keywords) # return is a list
        # Format search results:
        search_reply = '[ Video Search 视频搜索结果 ]' + '\n'
        if len(search_responses) == 0:
            search_reply += '[ Nill 无结果 ]'
        else:
            for i in range(len(search_responses)): search_reply += '\n' + str(search_responses[i])
        print ('')
        print (search_reply)
        return search_reply

In [29]:
itchat.run()

Start auto replying.
Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib64/python3.5/http/client.py", line 1174, in getresponse
    response.begin()
  File "/usr/lib64/python3.5/http/client.py", line 282, in begin
    version, status, reason = self._read_status()
  File "/usr/lib64/python3.5/http/client.py", line 251, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During 


Downloaded video file name is: 170607-201334.mp4
[INFOR] Start Video Analysis...


Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/itchat-1.3.4-py3.5.egg/itchat/components/register.py", line 60, in configured_reply
    r = replyFn(msg)
  File "<ipython-input-27-d76896c775bb>", line 10, in download_files
    parm_video_response = KudosData_VIDEO_DETECTION(encode_media(msg['FileName']))
  File "<ipython-input-8-b68882df4bd9>", line 13, in KudosData_VIDEO_DETECTION
    responses1 = request1.execute(num_retries=3)
  File "/home/user/env_py3/lib/python3.5/site-packages/oauth2client/_helpers.py", line 133, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "/home/user/env_py3/lib/python3.5/site-packages/googleapiclient/http.py", line 835, in execute
    method=str(self.method), body=self.body, headers=self.headers)
  File "/home/user/env_py3/lib/python3.5/site-packages/googleapiclient/http.py", line 162, in _retry_request
    resp, content = http.request(uri, method, *args, **kwargs)
  File "/home/user/env_py3/lib/pyt

[ Video Search 视频搜索结果 ]

{'confidence': 0.9878092, 'level': 'VIDEO_LEVEL', 'segment': {'startTimeOffset': '-1', 'endTimeOffset': '-1'}}
{'confidence': 0.9878092, 'level': 'SHOT_LEVEL', 'segment': {'endTimeOffset': '5499941'}}
[ Video Search 视频搜索结果 ]

{'confidence': 0.996535959432676, 'level': 'VIDEO_LEVEL', 'segment': {'startTimeOffset': '-1', 'endTimeOffset': '-1'}}
{'confidence': 0.996535959432676, 'level': 'SHOT_LEVEL', 'segment': {'endTimeOffset': '5499941'}}
[ Video Search 视频搜索结果 ]

{'confidence': 0.9779057, 'level': 'VIDEO_LEVEL', 'segment': {'startTimeOffset': '-1', 'endTimeOffset': '-1'}}
{'confidence': 0.9779057, 'level': 'SHOT_LEVEL', 'segment': {'endTimeOffset': '5499941'}}

Downloaded video file name is: 170607-201732.mp4
[INFOR] Start Video Analysis...


Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/itchat-1.3.4-py3.5.egg/itchat/components/register.py", line 60, in configured_reply
    r = replyFn(msg)
  File "<ipython-input-27-d76896c775bb>", line 10, in download_files
    parm_video_response = KudosData_VIDEO_DETECTION(encode_media(msg['FileName']))
  File "<ipython-input-8-b68882df4bd9>", line 13, in KudosData_VIDEO_DETECTION
    responses1 = request1.execute(num_retries=3)
  File "/home/user/env_py3/lib/python3.5/site-packages/oauth2client/_helpers.py", line 133, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "/home/user/env_py3/lib/python3.5/site-packages/googleapiclient/http.py", line 835, in execute
    method=str(self.method), body=self.body, headers=self.headers)
  File "/home/user/env_py3/lib/python3.5/site-packages/googleapiclient/http.py", line 162, in _retry_request
    resp, content = http.request(uri, method, *args, **kwargs)
  File "/home/user/env_py3/lib/pyt

...

[ Video Search 视频搜索结果 ]

{'confidence': 0.995469267801902, 'level': 'VIDEO_LEVEL', 'segment': {'startTimeOffset': '-1', 'endTimeOffset': '-1'}}
{'confidence': 0.995469267801902, 'level': 'SHOT_LEVEL', 'segment': {'endTimeOffset': '5499941'}}


Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib64/python3.5/http/client.py", line 1174, in getresponse
    response.begin()
  File "/usr/lib64/python3.5/http/client.py", line 282, in begin
    version, status, reason = self._read_status()
  File "/usr/lib64/python3.5/http/client.py", line 270, in _read_status
    raise BadStatusLine(line)
http.client.BadStatusLine: HTTP/1.1 0 -


During handling of the above exception, another exception occurred:

Traceback (most recent call last):

[ Video Search 视频搜索结果 ]
[ Nill 无结果 ]


Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib64/python3.5/http/client.py", line 1174, in getresponse
    response.begin()
  File "/usr/lib64/python3.5/http/client.py", line 282, in begin
    version, status, reason = self._read_status()
  File "/usr/lib64/python3.5/http/client.py", line 251, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above

Traceback (most recent call last):
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/user/env_py3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib64/python3.5/http/client.py", line 1174, in getresponse
    response.begin()
  File "/usr/lib64/python3.5/http/client.py", line 282, in begin
    version, status, reason = self._read_status()
  File "/usr/lib64/python3.5/http/client.py", line 251, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above

<font color='blue'>
<font color='blue'>
[提问] 是不是可以用图片来搜索视频内容？需要怎么处理？ 
[Question] Can we use image as input to search video content? How?
</font>


In [33]:
# interupt kernel, then logout
itchat.logout() # 安全退出

<ItchatReturnValue: {'BaseResponse': {'Ret': 0, 'RawMsg': 'logout successfully.', 'ErrMsg': '请求成功'}}>

### 恭喜您！已经完成了：
### 第五课：视频识别和处理
### Lesson 5: Video Recognition & Processing
* 识别视频的场景片段 (Detect shots change in video)
* 识别视频消息中的物体名字 (Recognize objects in video/shots)
* 直接搜索视频内容 (Search content in video)

### 下一课是:
### 第六课：互动智能机器人应用
### Lesson 6: Intelligent & Interactive Chat-bot Applications
* 多语言翻译器 (Language translator)
* 图文多媒体的订阅和点播 (Multi-media broadcast & on-demand subscription)
* 文章的概括和缩写 (Automatic article summary)
* 不良图片的识别；不良视频片段的自动识别和定位 (Explicit content detector: i.e. adult content or violent content)
* 基于商品图片的搜索和商家价格比较 (Best retail price finder using an image of goods)

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: left;">

